In [3]:
%%sh
pip install -qU httpx[http2] yfinance openai

In [31]:
import httpx
import json
import pprint
import yfinance
from openai import OpenAI
from IPython.display import Markdown
from api_key import api_key

endpoint="https://models.arcee.ai/v1"

client = OpenAI(
    base_url=endpoint,
    api_key=api_key,
    http_client=httpx.Client(http2=True)
)

In [3]:
def get_stock_price(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    price = stock.history(period="1d")["Close"].values[0]
    return f"The last closing price of {company_name} ({stock_symbol}) was ${price:.2f}."


def get_ceo_name(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    info = stock.info
    ceo = info['companyOfficers'][0]['name']
    return f"The CEO of {company_name} is {ceo}. The full job title is {info['companyOfficers'][0]['title']}."


def get_company_summary(company_name, stock_symbol):
    stock = yfinance.Ticker(stock_symbol)
    summary = stock.info['longBusinessSummary']
    return summary

In [ ]:
get_stock_price("Google", "GOOGL")

In [ ]:
get_ceo_name("Ford", "F")

In [ ]:
get_company_summary("Verizon", "VZ")

In [158]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": """Retrieves the most recent closing stock price that can answer questions about:
- Current stock price
- Recent stock performance
- Company market value
- Share price information

Use this function for questions like:
- "What's the stock price of [company]?"
- "How much does [company] stock cost?"
- "What's [company]'s current share price?"
- "How much are shares of [company] trading for?"
- "What's the latest price for [stock symbol]?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., 'Apple Inc.', 'Tesla, Inc.', 'McDonald's Corporation')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'AAPL', 'TSLA', 'MCD')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_ceo_name",
            "description": """Retrieves current leadership information that can answer questions about:
- CEO name and title
- Top executive information
- Company leadership
- Management details

Use this function for questions like:
- "Who is the CEO of [company]?"
- "Who runs [company]?"
- "What's the name of [company]'s chief executive?"
- "Who is in charge of [company]?"
- "Who is [company]'s leader?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., 'Microsoft Corporation', 'Meta Platforms, Inc.', 'Ford Motor Company')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'MSFT', 'META', 'F')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_company_summary",
            "description": """Retrieves a detailed business summary that can answer questions about:
- Products and services offered
- Target markets and industries served
- Manufacturing and production capabilities
- Business divisions and segments
- Geographic operations
- Company history and background
- Competitive position
- Core business activities

Use this function for questions like:
- "Does [company] make [product]?"
- "What industries does [company] serve?"
- "What are [company]'s main products?"
- "Does [company] operate in [industry/market]?"
- "What does [company] do?"
- "Tell me about [company]'s business"
- "What kind of company is [company]?"
""",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The full company name (e.g., '3M Company', 'Johnson & Johnson', 'The Coca-Cola Company')",
                    },
                    "stock_symbol": {
                        "type": "string",
                        "description": "The stock market ticker symbol (e.g., 'MMM', 'JNJ', 'KO')",
                    },
                },
                "required": ["company_name", "stock_symbol"],
            },
        },
    }
]

In [159]:
def call_tools(user_prompt, max_tokens=128):
    response = client.chat.completions.create(
        model="caller",
        messages=[

            {
                "role": "user",
                "content": user_prompt
            }
        ],
        tools=tools,
        tool_choice="auto",
        max_tokens=max_tokens,
    )
    
    tool_calls = response.choices[0].message.tool_calls

    # Check if there are any tool calls
    if tool_calls:
        # Extract the first tool call (assuming there's at least one)
        first_tool_call = tool_calls[0]
        
        # Extract function name and arguments
        function_name = first_tool_call.function.name
        arguments_json = first_tool_call.function.arguments
        arguments_dict = json.loads(arguments_json)
        
        ## Assuming the function is in the current namespace or imported
        if function_name in globals():
            # Get the function object based on its name
            function_to_call = globals()[function_name]
            
            print(f"Calling {function_name} with arguments: {arguments_dict}")
            
            # Call the function with unpacked keyword arguments
            result = function_to_call(**arguments_dict)
            return result
        else:
            print(f"Function {function_name} not found in the global namespace.")
            return None
    else:
        # No tool call: print the generated response
        print("No tool call")
        return None


In [247]:
def call_tools_and_invoke_model(user_prompt, max_tokens=1024):
    tool_result = call_tools(user_prompt)
    
    response = client.chat.completions.create(
        model="virtuoso-small",
        messages=[
            {
                "role": "system",
                "content": """You are an expert financial and business analyst providing precise, well-researched answers.

Your capabilities:
- Analyze company information, financials, and market data
- Compare companies and their products/services
- Provide insights about business strategies and market positions
- Explain industry trends and competitive dynamics

Guidelines:
1. Always start with analyzing the tool result if available
2. Structure your response clearly using markdown
3. Be concise but thorough
4. Include relevant context from your knowledge
5. Support claims with data and sources when possible
6. Stay neutral and objective

Required sections:
* Tool information - Did you use the tool or not?
* Direct Answer - Address the main question first.
* Analysis - Provide context and deeper insights
* Resources - if applicable, include for all companies:
   - Top-level company website
   - Yahoo Finance page (if the company is listed)
   - URL for company SEC filings (if the company is publicly traded)
   - URL for Google search based on the user question

If the tool result is unavailable or insufficient, acknowledge this first and proceed with your best analysis based on available knowledge."""
            },
            {
                "role": "user",
                "content": f"""
                    question: {user_prompt}
                    tool result: {tool_result}
                """
            }
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

In [251]:
#user_prompt = "Who's the CEO of General Motors?"
user_prompt = "What's the last price of Mc Donalds?"
#user_prompt = "Does 3M make filtration products for the automotive industry?"
#user_prompt = "On which products do Procter and Gamble and Johnson & Johnson compete the most?"
#user_prompt = "What's the population of the capital of New Zealand?"
#user_prompt = "Is Lamb Weston the largest producer of french fries?"

In [ ]:
response = call_tools(user_prompt)
pprint.pprint(response)

In [ ]:
response = call_tools_and_invoke_model(user_prompt)
Markdown(response)